# Notebook de conception de Notebook

Ce Notebook .Net interactive a pour objectif de permettre la création assistée d'autres notebooks .Net interactive en confiant le soin à ChatGPT d'analyser et de proposer des modifications d'une version courante, et en prenant en charge la mise à jour et l'exécution des mises à jour en function calling Open AI grâce à l'API .Net interactive. 


### 1. Initialisation

On installe des packages pour la manipulation de notebook et pour l'orchestration de LLMs.

In [ ]:
#pragma warning disable SKEXP0060
#!import Semantic-kernel-AutoInteractive.ipynb

### 5. Reccueil des informations

Selon le mode de fourniture des informations choisi, on récupère la tâche à accomplir dans le notebook de travail.

In [ ]:
var infoCollectionDisplay = display("Collecte d'informations en cours...");

var mode = InformationMode.Variable;

string taskDescription = "Créer un notebook .Net interactive permettant de requêter DBPedia. Utilisant le package Nuget dotNetRDF. Utiliser également Plotly ou Xplot pour afficher un graphique synthétisant les résultats. Attention le package Nuget VDS.RDF n'existe pas, ne l'hallucine pas, il s'agit du namespace.";

if (mode == InformationMode.Variable)
{
    display("Utilisation de la variable pour la description de la tâche.");
}
else if (mode == InformationMode.Prompt)
{
    var questions = new[]
    {
        "Bonjour! Veuillez fournir une brève description de la tâche à accomplir.",
        "Quels sont les principaux objectifs de cette tâche?",
        "Y a-t-il des contraintes ou des conditions spécifiques à prendre en compte?",
        "Des informations supplémentaires que vous souhaitez ajouter?"
    };

    taskDescription = string.Empty;
    foreach (var question in questions)
    {
        var response = await IKernel.GetInputAsync(question);
        taskDescription += $"{question}\\n{response}\\n\\n";
    }
}


display("Informations recueillies :\\n" + taskDescription);

### 6. Exécution et Mise à Jour Itérative

In [ ]:
var notebookPath = @$"./Workbooks/Workbook-{DateTime.Now.ToFileTime()}.ipynb";
// var notebookPath = @$"./Workbooks/Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}.ipynb";
SetStartingNotebook(taskDescription, notebookPath);

var logger = new DisplayLogger("NotebookUpdater", LogLevel.Trace);
var nbIterations = 10;
display("Création de l'instance NotebookUpdater...");
var updater = new NotebookUpdater(semanticKernel, notebookPath, nbIterations, logger);

display("Appel à UpdateNotebook...");
var response = await updater.UpdateNotebook();

display($"Résultat de l'exécution du notebook :\n{response}");


### Conclusion